In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open("names.txt", "r").read().splitlines()


In [3]:
charset = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(charset)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [4]:
block_size = 3

def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))
Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [5]:
def cmp(s,dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f"{s:15s}: exact: {str(ex):5s} approx: {str(app):5s} maxdiff: {maxdiff}")

In [6]:
n_embd = 10 # embedding dimension
n_hidden = 64 # number of neurons in the hidden layer

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

bngain = torch.ones((1, n_hidden))*0.1 + 1.0
bnbias = torch.zeros((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad = True

In [7]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [8]:
# Forward pass
emb = C[Xb] 
embcat = emb.view(emb.shape[0], -1)

hprebn = embcat @ W1 + b1
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1) * bndiff2.sum(0, keepdim=True)
bnvarinv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvarinv
hpreact = bngain * bnraw + bnbias
h = torch.tanh(hpreact)
logits = h @ W2 + b2
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# Backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvarinv, bnvar, bndiff2, bndiff, bnmeani, hprebn,
          embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.3482, grad_fn=<NegBackward0>)

In [9]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n
cmp('dlogprobs', dlogprobs, logprobs)

dlogprobs      : exact: True  approx: True  maxdiff: 0.0


In [10]:
dprobs = 1/probs * dlogprobs
cmp('dprobs', dprobs, probs)

dprobs         : exact: True  approx: True  maxdiff: 0.0


In [11]:
dcount_sum_inv = (counts *  dprobs).sum(1, keepdim=True)  
cmp('dcount_sum_inv', dcount_sum_inv, counts_sum_inv)

dcount_sum_inv : exact: True  approx: True  maxdiff: 0.0


In [12]:
dcounts = counts_sum_inv * dprobs
# update this further down

In [13]:
dcounts_sum = - (counts_sum**-2) * (dcount_sum_inv)
cmp('dcounts_sum', dcounts_sum, counts_sum)

dcounts_sum    : exact: True  approx: True  maxdiff: 0.0


In [14]:
dcounts += torch.ones_like(counts) * dcounts_sum
cmp('dcounts', dcounts, counts)

dcounts        : exact: True  approx: True  maxdiff: 0.0


In [15]:
dnorm_logits = counts * dcounts
cmp('dnorm_logits', dnorm_logits, norm_logits)

dnorm_logits   : exact: True  approx: True  maxdiff: 0.0


In [16]:
dlogits = dnorm_logits.clone()

dlogit_maxes = - dnorm_logits.sum(1, keepdim=True)
cmp('dlogit_maxes', dlogit_maxes, logit_maxes)

dlogit_maxes   : exact: True  approx: True  maxdiff: 0.0


In [17]:
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
cmp('dlogits', dlogits, logits)

dlogits        : exact: True  approx: True  maxdiff: 0.0


In [18]:
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)

cmp('dh', dh, h)
cmp('dW2', dW2, W2)
cmp('db2', db2, b2)

dh             : exact: True  approx: True  maxdiff: 0.0
dW2            : exact: True  approx: True  maxdiff: 0.0
db2            : exact: True  approx: True  maxdiff: 0.0


In [19]:
dhpreact = (1 - h**2) * dh
cmp('dhpreact', dhpreact, hpreact)

dhpreact       : exact: True  approx: True  maxdiff: 0.0


In [20]:
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)
cmp('dbngain', dbngain, bngain)
cmp('dbnraw', dbnraw, bnraw)
cmp('dbnbias', dbnbias, bnbias)

dbngain        : exact: True  approx: True  maxdiff: 0.0
dbnraw         : exact: True  approx: True  maxdiff: 0.0
dbnbias        : exact: True  approx: True  maxdiff: 0.0


In [ ]:
dbndiff = bnvarinv * dbnraw
dbnvarinv = (bndiff * dbnraw).sum(0, keepdim=True)
dbnvar = -0.5 * (bnvar + 1e-5)**-1.5 * dbnvarinv 
# wait until later calculation of the other branch cmp('dbndiff', dbndiff, bndiff)
cmp('dbnvarinv', dbnvarinv, bnvarinv)   

dbndiff        : exact: False approx: False maxdiff: 0.0011080314870923758
dbnvarinv      : exact: True  approx: True  maxdiff: 0.0


In [30]:
dbndiff2 = torch.ones_like(bndiff2) * (1/(n-1) * dbnvar)
cmp('dbndiff2', dbndiff2, bndiff2)

dbndiff2       : exact: True  approx: True  maxdiff: 0.0


In [31]:
dbndiff += 2 * bndiff * dbndiff2
cmp('dbndiff', dbndiff, bndiff)

dbndiff        : exact: True  approx: True  maxdiff: 0.0


In [ ]:
dhprebn = dbndiff.clone()
dbnmeani = - torch.ones_like(bnmeani) * dbndiff.sum(0, keepdim=True)
# later calculation cmp('dhprebn', dhprebn, hprebn)
cmp('dbnmeani', dbnmeani, bnmeani)

dhprebn        : exact: False approx: False maxdiff: 0.001130998833104968
dbnmeani       : exact: True  approx: True  maxdiff: 0.0


In [33]:
dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)
cmp('dhprebn', dhprebn, hprebn)

dhprebn        : exact: True  approx: True  maxdiff: 0.0


In [34]:
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
cmp('dembcat', dembcat, embcat)
cmp('dW1', dW1, W1)
cmp('db1', db1, b1)

dembcat        : exact: True  approx: True  maxdiff: 0.0
dW1            : exact: True  approx: True  maxdiff: 0.0
db1            : exact: True  approx: True  maxdiff: 0.0
